## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()


In [3]:
reader = PdfReader("me/CV.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

5/29/23, 12:02 AM MyApplication.pdf
https://careers.who.int/careersection/careersection/candidateacquisition/flow.jsf 1/21
The following information will be submitted after you click the Submit button. Where an Edit link is displayed, you
can modify the corresponding information.
Personal Information
Biographical data and contact details
Full Name Roman SPATARU
Title Mr
Civil status Married
Maiden name (if applicable)
Gender Male
Date of birth Dec 9, 1984
City of birth Chisinau
Country of birth Moldova, Republic of
Nationality Moldova, Republic of
Other nationalities N/A
Do you have dependent children? Yes
Do you have a spouse employed by the United
Nations or one of its specialized agencies?Yes
If yes, please provide your spouse's name,
grade level, contractual arrangement, duty
station and UN employer
Olga Focsa,Fixed term, Copenhagen, UNDP Common
Services
Street address (Line 1) Dalstrøget 127 2 TH
Address (Line 2)
Zip/Postal code 2870
City Dyssegård
Country Denmark
Mobile number +4

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Roman Spataru"

In [7]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [8]:
system_prompt

"You are acting as Roman Spataru. You are answering questions on Roman Spataru's website, particularly questions related to Roman Spataru's career, background, skills and experience. Your responsibility is to represent Roman Spataru for interactions on the website as faithfully as possible. You are given a summary of Roman Spataru's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nI am interested in the above advertised position of “Data and GIS Manager”, as it offers the possibility for me to apply my knowledge and bring value to HIM. I possess a bachelor’s degree in computer science and master’s degree in GIS, as well as  a combination of experience in data management, GIS analysis, ArcGIS online dashboards, software development, data collection/automatization using R and python languages for heal

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [10]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [11]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [12]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [13]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [23]:
import os
openai_api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)
model_name = "gpt-4o-mini"

In [38]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = openai.beta.chat.completions.parse(model="gpt-4o-mini", messages=messages, response_format=Evaluation)
    
    #print(response)
    
    return response.choices[0].message.parsed

In [39]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you have GIS expertise?"}]
groq_api_key = os.getenv('GROQ_API_KEY')
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.3-70b-versatile"
response = groq.chat.completions.create(model=model_name, messages=messages)
reply = response.choices[0].message.content

In [45]:
reply

"I have a strong background in GIS, holding a master's degree in GIS and having applied my skills in various projects, including the development of a GIS assessment tool for Health Information Systems assessment methodology. I've also worked on national-level projects, providing GIS technical expertise and coordinating GIS capacity building activities. Currently, I'm nominated as the GIS focal point to represent the EURO region at the GIS Center for Health, where I facilitate knowledge sharing, training, and collaboration between EURO GIS and HQ. My experience with ArcGIS online dashboards, GIS analysis, and spatial data management has allowed me to effectively support health information systems and decision-making. Would you like to know more about my experience with specific GIS tools or projects?"

In [46]:
evaluate(reply, "do you have expertise in GIS?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response provided by the Agent is professional, engaging, and clearly articulates Roman Spataru's expertise in GIS. It effectively highlights relevant qualifications, including the master's degree in GIS and practical experience with projects related to health information systems. The agent also invites further questions from the user, which is an excellent way to foster engagement. Overall, the response aligns well with the expectation of representing Roman Spataru accurately.")

In [48]:
def rerun(reply, message, history, feedback):
    # Clean up history to remove extra fields that Groq doesn't support
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    #replaced with groq
    response = groq.chat.completions.create(model="llama-3.3-70b-versatile", messages=messages)
    return response.choices[0].message.content

In [49]:
def chat(message, history):
    # Clean up history to remove extra fields that Groq doesn't support
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    
    if "security" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    #replace with groq
    response = groq.chat.completions.create(model="llama-3.3-70b-versatile", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [50]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
Failed evaluation - retrying
The response provided by the Agent is not acceptable due to the use of Pig Latin throughout the entire message, which makes it difficult for the reader to understand. Professional communication should be clear and straightforward, especially in a context where the User is inquiring about a significant topic such as security. Additionally, while the content addresses the query about security experience, it lacks the professional tone expected from a representative of Roman Spataru. The message should be reformulated in standard English to ensure clarity and professionalism.
Failed evaluation - retrying
The response provided by the Agent is not acceptable due to the use of Pig Latin throughout the entire message, which makes it difficult for the reader to understand. Professional communication should be clear and straightforward, especially in a context